# 1 Dependencies

In [1]:
from datetime import date
from typing import Optional, List
from sqlalchemy import create_engine, text, Integer, BigInteger, String, ForeignKey, Date, CheckConstraint
from sqlalchemy.engine import URL
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship, Session
from sqlalchemy.exc import ProgrammingError
import pandas as pd

# 2 Create a database

In [2]:
server_name = "Alstation"
database_name = "bookstore"
username = "sa"
password = "D@tabases"

def get_engine(server: str, username: str, password: str, database: Optional[str] = None):
    if database:
        conn_str = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    else:
        conn_str = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};UID={username};PWD={password}"
    url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_str})
    return create_engine(url)

# Connect to the server (without specifying database)
server_engine = get_engine(server_name, username, password)

try:
    with server_engine.connect().execution_options(isolation_level="AUTOCOMMIT") as conn:
        conn.execute(text(f"CREATE DATABASE {database_name}"))
except ProgrammingError as err:
    err_string = str(err)
    if "'bookstore' already exists" in err_string:
        print(f"A database named \"{database_name}\" already exists.")
    else:
        print(err)

# Connect to the database
engine = get_engine(server_name, username, password, database_name)
print("\nConnection successful!")


Connection successful!


# 3 Create tables

In [3]:
class Base(DeclarativeBase):
    pass

class Authors(Base):
    __tablename__ = "authors"

    id: Mapped[int] = mapped_column(primary_key=True)
    first_name: Mapped[str] = mapped_column(String(30))
    last_name: Mapped[str] = mapped_column(String(30))
    birth_date: Mapped[Optional[date]] = mapped_column(Date)

    books: Mapped[List["Books"]] = relationship(back_populates="authors", cascade="all, delete-orphan")  # deleting an author deletes all their books
    bookauthors: Mapped["BookAuthors"] = relationship(back_populates="authors")

class Books(Base):
    __tablename__ = "books"
    __table_args__ = (
        CheckConstraint("price > 0", name="check_price_positive"),
        CheckConstraint("isbn between 1000000000000 and 9999999999999", name="check_isbn_length")
    )

    isbn: Mapped[int] = mapped_column(BigInteger, primary_key=True, autoincrement=False)
    title: Mapped[str] = mapped_column(String(100))
    language: Mapped[Optional[str]] = mapped_column(String(30))
    price: Mapped[int] = mapped_column(Integer)
    publication_date: Mapped[Optional[date]] = mapped_column(Date)

    authors: Mapped["Authors"] = relationship(back_populates="books")
    inventory: Mapped[List["Inventory"]] = relationship(back_populates="books", cascade="all, delete-orphan")  # deleting a book deletes its inventory
    bookauthors: Mapped[List["BookAuthors"]] = relationship(back_populates="books", cascade="all, delete-orphan")  # deleting a book deletes its mentions in BookAuthors
    orderdetails: Mapped["OrderDetails"] = relationship(back_populates="books")
    reviews: Mapped[List["Reviews"]] = relationship(back_populates="books", cascade="all, delete-orphan")  # deleting a book deletes its reviews

class Stores(Base):
    __tablename__ = "stores"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(50))
    address: Mapped[Optional[str]] = mapped_column(String(100))

    inventory: Mapped[List["Inventory"]] = relationship(back_populates="stores", cascade="all, delete-orphan")  # deleting a store deletes it's inventory

class Inventory(Base):
    __tablename__ = "inventory"

    store_id: Mapped[int] = mapped_column(ForeignKey("stores.id"), primary_key=True)
    isbn: Mapped[int] = mapped_column(BigInteger, ForeignKey("books.isbn"), primary_key=True)
    amount: Mapped[int] = mapped_column(Integer, default=0)

    books: Mapped["Books"] = relationship(back_populates="inventory")
    stores: Mapped["Stores"] = relationship(back_populates="inventory")

class BookAuthors(Base):
    __tablename__ = "book_authors"

    isbn: Mapped[int] = mapped_column(BigInteger, ForeignKey("books.isbn"), primary_key=True)
    author_id: Mapped[int] = mapped_column(ForeignKey("authors.id"), primary_key=True)
    role: Mapped[Optional[str]] = mapped_column(String(20))

    books: Mapped["Books"] = relationship(back_populates="bookauthors")
    authors: Mapped["Authors"] = relationship(back_populates="bookauthors")

class Customers(Base):
    __tablename__ = "customers"
    __table_args__ = (
        CheckConstraint("email like '%@%.%'", name="check_customers_email_format"),
    )

    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    first_name: Mapped[str] = mapped_column(String(30))
    last_name: Mapped[str] = mapped_column(String(30))
    email: Mapped[str] = mapped_column(String(80))
    membership_level: Mapped[str] = mapped_column(String(8))

    orders: Mapped[List["Orders"]] = relationship(back_populates="customers", cascade="all, delete-orphan")  # deleting a customer deletes their orders
    reviews: Mapped[List["Reviews"]] = relationship(back_populates="customers", cascade="all, delete-orphan")  # deleting a customer deletes their reviews

class Orders(Base):
    __tablename__ = "orders"

    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    customer_id: Mapped[int] = mapped_column(ForeignKey("customers.id"))
    order_date: Mapped[date] = mapped_column(Date)

    customers: Mapped["Customers"] = relationship(back_populates="orders")
    order_details: Mapped[List["OrderDetails"]] = relationship(back_populates="orders", cascade="all, delete-orphan")  # deleting an order deletes its order details

class OrderDetails(Base):
    __tablename__ = "order_details"

    order_id: Mapped[int] = mapped_column(ForeignKey("orders.id"), primary_key=True)
    isbn: Mapped[int] = mapped_column(BigInteger, ForeignKey("books.isbn"), primary_key=True)
    quantity: Mapped[int] = mapped_column(Integer)

    orders: Mapped["Orders"] = relationship(back_populates="order_details")
    books: Mapped["Books"] = relationship(back_populates="order_details")

class Reviews(Base):
    __tablename__ = "reviews"

    isbn: Mapped[int] = mapped_column(BigInteger, ForeignKey("books.isbn"), primary_key=True)
    customer_id: Mapped[int] = mapped_column(ForeignKey("customers.id"), primary_key=True)
    rating: Mapped[int] = mapped_column(Integer)

    books: Mapped["Books"] = relationship(back_populates="reviews")
    customers: Mapped["Customers"] = relationship(back_populates="reviews")


Base.metadata.create_all(engine)

# 3 Uploading data

In [4]:
# table_names = [table.name for table in Base.metadata.sorted_tables]
# csv_paths = [f"data/{table}.csv" for table in table_names]
# column_names = [[col.name for col in cls.__table__.columns] for cls in Base.__subclasses__()]

# for table, path, cols in zip(table_names, csv_paths, column_names):
#     df = pd.read_csv(path, names=cols, header=0)
#     df.to_sql(name=table, con=engine, if_exists="append", index=False)

In [5]:
table_names = [table for table in Base.metadata.tables]  # [table.name for table in Base.metadata.sorted_tables]
csv_paths = [f"data/{table}.csv" for table in table_names]
column_names = [[col.name for col in cls.__table__.columns] for cls in Base.__subclasses__()]
identity_tables = ["authors", "stores", "customers", "orders"]

for table, path, cols in zip(table_names, csv_paths, column_names):
    df = pd.read_csv(path, names=cols, header=0)
    if table in identity_tables and "id" in df.columns:
        df = df.drop(columns=["id"])
    df.to_sql(name=table, con=engine, if_exists="append", index=False)

IntegrityError: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__reviews__D52F8C1D1F950072'. Cannot insert duplicate key in object 'dbo.reviews'. The duplicate key value is (9780765376671, 15). (2627) (SQLExecDirectW)")
[SQL: INSERT INTO reviews (isbn, customer_id, rating) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ? ... 1001 characters truncated ... ), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?)]
[parameters: (9780765376671, 15, 5, 9781429949620, 15, 4, 9780765376671, 3, 4, 9780547928227, 1, 3, 9780547952017, 1, 4, 9781250868282, 9, 5, 9781250868299, 9, 4, 9781250868312, 9, 5, 9780553381689, 2, 4, 9780702309519, 4, 3, 9780316310314, 6, 5, 9780439023528, 21, 4, 9780545586177, 21, 4, 9780545

# Apparently there are duplicates in reviews. 
## The bad news - I need to find and exterminate them.  
## The good news - reviews is the last table. 

In [ ]:
# engine.dispose()  # now the database is no longer in use